In [206]:
reference_sequence = "LKSDNVOEBOICSKJDNOJEVNSDKLCX"
pdb_sequence = [('D.34.C', 'K'), ('D.34.D', 'S'), ('D.34.D', 'D'), ('D.34.E', 'N'), ('D.35.', 'V'), ('D.35.A', 'O'), ('D.36.', 'O'), ('D.40.', 'X')]

In [207]:
def convert_loc_to_idx(pdb_tuple):
    pos = pdb_tuple[0].split(".")
    idx = int(pos[1])
    return idx

In [208]:
import re

seq_idx = []
align_int = 0
for i, (pos, amino) in enumerate(pdb_sequence):
    idx = convert_loc_to_idx((pos, amino)) #+ align_int
    if pos.split(".")[-1] != "":
        # align_int += 1
        if len(seq_idx) > 0:
            seq_idx[-1] += (amino,)
        else:
            seq_idx.append((idx, amino))
    else:
        seq_idx.append((idx, amino))

In [209]:
seq_idx
# [(142, 'K', 'S', 'D'), (146, 'N'), (148, 'O')]

[(34, 'K', 'S', 'D', 'N'), (35, 'V', 'O'), (36, 'O'), (40, 'X')]

In [210]:
# reference_sequence = "MMAAGHSFDJNDKVDKNKNDLNDFLVKNDFKNVKNXFVSSDNDOSRJIODRJ"
# pdb_sequence = [('A.254.', 'A'), ('A.255.', 'G'), ('A.258.', 'F'), ('A.258.A', 'D'), ('A.264.', 'D')]
        # expected_result = [
        #     [('A.254.', 2), ('A.255.', 4), ('A.258.', 7), ('A.258.A', 8), ('A.264.', 14)],

In [211]:
reg = r""
gaps = []
for i, tup in enumerate(seq_idx):
    if i == 0:
        for amino in tup[1:]:
            reg += re.escape(amino)
            gaps.append(0)
        # gaps = gaps[1:]
        
    else: 
        # we don't know exactly how many aminos may be between position 1 and 2,
        # (because there could be N insertion aminos)
        # but we know it must be at least 1
        gap_min = tup[0] - seq_idx[i-1][0] -1    #+ (len(seq_idx[i-1]) + 1)
        gaps.append(gap_min)

        # if the aminos are in an insertion group, 
        # they come directly after each other
        if len(tup) > 2:
            # gap_min += 
            exact = re.escape(tup[1])
            for amino in tup[2:]:
                exact += re.escape(amino)
                gaps.append(0)
        else:
            exact = re.escape(tup[1])
    
        # we add the "?" due to *greediness*: https://stackoverflow.com/a/16619508
        # we want the regex to search from shortest to longest string, not longest to shortest
        reg += r"[A-Z]{" + re.escape(str(gap_min)) + r",}?" + re.escape(exact)

        
        
print("regular expression: ", reg)
print("gaps: ", gaps)
# KSD[A-Z]{1,}?N[A-Z]{1,}?O

regular expression:  KSDN[A-Z]{0,}?VO[A-Z]{0,}?O[A-Z]{3,}?X
gaps:  [0, 0, 0, 0, 0, 0, 0, 3]


In [212]:
# reference_sequence = "LKSDNVOEBOICSKJDNOJEVNSDKLCX"
# pdb_sequence = [('D.34.C', 'K'), ('D.34.D', 'S'), ('D.34.D', 'D'), ('D.34.E', 'N'), ('D.35.', 'V'), ('D.35.A', 'O'), ('D.36.', 'O'), ('D.40.C', 'X')]
# reg = A[A-Z]{0,}?G[A-Z]{2,}?FD[A-Z]{5,}?D
res = re.search(reg, reference_sequence)
print(res.group())

KSDNVOEBOICSKJDNOJEVNSDKLCX


In [213]:
match_start = res.start()
match_start

1

In [214]:
seq_idx

[(34, 'K', 'S', 'D', 'N'), (35, 'V', 'O'), (36, 'O'), (40, 'X')]

In [215]:
match_idx = [(i+match_start, c) for i, c in enumerate(res.group())]
match_idx

[(1, 'K'),
 (2, 'S'),
 (3, 'D'),
 (4, 'N'),
 (5, 'V'),
 (6, 'O'),
 (7, 'E'),
 (8, 'B'),
 (9, 'O'),
 (10, 'I'),
 (11, 'C'),
 (12, 'S'),
 (13, 'K'),
 (14, 'J'),
 (15, 'D'),
 (16, 'N'),
 (17, 'O'),
 (18, 'J'),
 (19, 'E'),
 (20, 'V'),
 (21, 'N'),
 (22, 'S'),
 (23, 'D'),
 (24, 'K'),
 (25, 'L'),
 (26, 'C'),
 (27, 'X')]

In [216]:
# pdb_sequence = [('C.140.B', 'K'), ('C.140.C', 'S'), ('C.140.D', 'D'), ('C.143.', 'N'), ('C.145.', 'O')]
k = 0
out = [(pdb_sequence[k][0], match_idx[k][0])]

i = k = 1
while len(out) != len(pdb_sequence):
# for tup in match_idx[1:]:
    print(out)
    if match_idx[i][1] == pdb_sequence[k][1]:
        print(f"matching : {match_idx[i][1]} == {pdb_sequence[k][1]}")

        if (match_idx[i][0] - out[k-1][1] ) > (gaps[k]):
            print(f"({match_idx[i][0]} - {out[k-1][1]} ) > gap ({gaps[k]})")
            out.append((pdb_sequence[k][0], match_idx[i][0]))
            k += 1
    i+=1

out

[('D.34.C', 1)]
matching : S == S
(2 - 1 ) > gap (0)
[('D.34.C', 1), ('D.34.D', 2)]
matching : D == D
(3 - 2 ) > gap (0)
[('D.34.C', 1), ('D.34.D', 2), ('D.34.D', 3)]
matching : N == N
(4 - 3 ) > gap (0)
[('D.34.C', 1), ('D.34.D', 2), ('D.34.D', 3), ('D.34.E', 4)]
matching : V == V
(5 - 4 ) > gap (0)
[('D.34.C', 1), ('D.34.D', 2), ('D.34.D', 3), ('D.34.E', 4), ('D.35.', 5)]
matching : O == O
(6 - 5 ) > gap (0)
[('D.34.C', 1), ('D.34.D', 2), ('D.34.D', 3), ('D.34.E', 4), ('D.35.', 5), ('D.35.A', 6)]
[('D.34.C', 1), ('D.34.D', 2), ('D.34.D', 3), ('D.34.E', 4), ('D.35.', 5), ('D.35.A', 6)]
[('D.34.C', 1), ('D.34.D', 2), ('D.34.D', 3), ('D.34.E', 4), ('D.35.', 5), ('D.35.A', 6)]
matching : O == O
(9 - 6 ) > gap (0)
[('D.34.C', 1), ('D.34.D', 2), ('D.34.D', 3), ('D.34.E', 4), ('D.35.', 5), ('D.35.A', 6), ('D.36.', 9)]
[('D.34.C', 1), ('D.34.D', 2), ('D.34.D', 3), ('D.34.E', 4), ('D.35.', 5), ('D.35.A', 6), ('D.36.', 9)]
[('D.34.C', 1), ('D.34.D', 2), ('D.34.D', 3), ('D.34.E', 4), ('D.35.', 

[('D.34.C', 1),
 ('D.34.D', 2),
 ('D.34.D', 3),
 ('D.34.E', 4),
 ('D.35.', 5),
 ('D.35.A', 6),
 ('D.36.', 9),
 ('D.40.', 27)]